In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 1. 加载你的色谱柱数据（替换为GitHub仓库中的相对路径）
df = pd.read_excel('/workspaces/pyGAPS/docs/examples/data/色谱柱1和色谱柱2在30度下分开做pca分析.xlsx')

# 2. 确认数据结构（关键！需根据你的实际列名调整）
print("=== 数据核心结构 ===")
print(f"数据形状：{df.shape}")
print(f"所有列名：{df.columns.tolist()}")
print("\n前5行数据：")
print(df.head())

# 3. 手动定义核心列（必须根据你的Excel列名修改！）
TARGET_COL = '保留因子'          # 目标列：要预测的保留因子
GROUP_COL = '色谱柱类型'         # 分组列：区分“色谱柱1”“色谱柱2”
# 特征列：排除目标列、分组列、无意义列（如样本ID）
FEATURE_COLS = [col for col in df.columns if col not in [TARGET_COL, GROUP_COL, '样本ID']]

print(f"\n=== 核心列定义（请确认是否匹配你的数据） ===")
print(f"特征列（输入）：{FEATURE_COLS}")
print(f"目标列（预测）：{TARGET_COL}")
print(f"分组列（色谱柱）：{GROUP_COL}")
print(f"色谱柱类型分布：\n{df[GROUP_COL].value_counts()}")

=== 数据核心结构 ===
数据形状：(8, 19)
所有列名：['样本编号', '苯_t0', '苯_tR', '苯_k', '甲苯_t0', '甲苯_tR', '甲苯_k', '乙苯_t0', '乙苯_tR', '乙苯_k', '丙苯_t0', '丙苯_tR', '丙苯_k', '丁苯_t0', '丁苯_tR', '丁苯_k', '戊苯_t0', '戊苯_tR', '戊苯_k']

前5行数据：
          样本编号   苯_t0    苯_tR     苯_k  甲苯_t0   甲苯_tR    甲苯_k  乙苯_t0   乙苯_tR  \
0  M85-15_色谱柱1  2.978  3.6277  0.2182  2.978  3.7997  0.2759  2.978  3.9555   
1  M85-15_色谱柱2  2.965  3.7180  0.2540  2.965  3.9110  0.3191  2.965  4.0860   
2  M80-20_色谱柱1  2.928  3.9683  0.3553  2.928  4.2956  0.4671  2.928  4.6170   
3  M80-20_色谱柱2  2.937  4.0140  0.3667  2.937  4.3390  0.4774  2.937  4.6600   
4  M75-25_色谱柱1  2.945  4.2219  0.4336  2.945  4.6723  0.5865  2.945  5.1502   

     乙苯_k  丙苯_t0   丙苯_tR    丙苯_k  丁苯_t0   丁苯_tR    丁苯_k  戊苯_t0   戊苯_tR    戊苯_k  
0  0.3282  2.978  4.1584  0.3964  2.978  4.4644  0.4991  2.978  4.8089  0.6148  
1  0.3781  2.965  4.3150  0.4553  2.965  4.6570  0.5707  2.965  5.0420  0.7005  
2  0.5768  2.928  5.0796  0.7348  2.928  5.7628  0.9682  2.928  6.5809  1.2476 

KeyError: '色谱柱类型'

In [5]:
import rdkit
from rdkit import Chem
from rdkit.Chem import Descriptors

print("RDKit 版本：", rdkit.__version__)

ModuleNotFoundError: No module named 'rdkit'

In [2]:
# 打印数据所有列名（关键！找到正确的列名）
print("=== 你的数据实际列名列表 ===")
for idx, col in enumerate(df.columns):
    print(f"列{idx+1}：{col}")

# 同时打印前几行数据，直观确认列名对应的值
print("\n=== 数据前5行（看列名对应的值） ===")
print(df.head())

=== 你的数据实际列名列表 ===


NameError: name 'df' is not defined

In [8]:
# ========== 关键修改：替换成你的实际列名 ==========
# 目标列：要预测的保留因子（替换成你数据里的保留因子列名）
TARGET_COL = '保留因子'  # 比如你的列名是'k值'，就改成 'k值'
# 分组列：区分色谱柱1/2的列（替换成你数据里的列名，比如'色谱柱'/'柱型'）
GROUP_COL = '色谱柱'     # 重点！把这里改成你数据里的列名（比如'柱型'）

# 特征列：排除目标列、分组列、无意义列（如样本ID）
FEATURE_COLS = [col for col in df.columns if col not in [TARGET_COL, GROUP_COL, '样本ID']]

print(f"\n=== 修正后的核心列定义 ===")
print(f"特征列（输入）：{FEATURE_COLS}")
print(f"目标列（预测）：{TARGET_COL}")
print(f"分组列（色谱柱）：{GROUP_COL}")

# 现在再打印色谱柱分布（不会报错了）
print(f"\n=== 色谱柱类型分布 ===")
print(df[GROUP_COL].value_counts())


=== 修正后的核心列定义 ===
特征列（输入）：['样本编号', '苯_t0', '苯_tR', '苯_k', '甲苯_t0', '甲苯_tR', '甲苯_k', '乙苯_t0', '乙苯_tR', '乙苯_k', '丙苯_t0', '丙苯_tR', '丙苯_k', '丁苯_t0', '丁苯_tR', '丁苯_k', '戊苯_t0', '戊苯_tR', '戊苯_k']
目标列（预测）：保留因子
分组列（色谱柱）：色谱柱

=== 色谱柱类型分布 ===


KeyError: '色谱柱'

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import joblib
import warnings
warnings.filterwarnings('ignore')
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# ========== 1. 加载数据（已修正路径） ==========
# 替换成你的实际路径（相对路径/绝对路径）
df = pd.read_excel('色谱柱1和色谱柱2在30度下分开做pca分析.xlsx', engine='openpyxl')

# ========== 2. 先查看实际列名（必做！） ==========
print("=== 你的数据实际列名 ===")
for idx, col in enumerate(df.columns):
    print(f"{idx+1}. {col}")
print("\n=== 数据前5行 ===")
print(df.head())

# ========== 3. 修正列名定义（重点！替换成你的实际列名） ==========
# 👇👇👇 这里一定要改！👇👇👇
TARGET_COL = '保留因子'    # 替换成你数据里的保留因子列名（比如'k值'/'保留因子k'）
GROUP_COL = '色谱柱'       # 替换成你数据里区分色谱柱1/2的列名（比如'柱型'/'柱子类型'）

# 特征列：自动排除目标列、分组列、无意义列（如样本ID）
FEATURE_COLS = [col for col in df.columns if col not in [TARGET_COL, GROUP_COL, '样本ID']]

# ========== 4. 验证列名是否正确 ==========
print(f"\n=== 修正后的列名配置 ===")
print(f"特征列：{FEATURE_COLS}")
print(f"目标列（保留因子）：{TARGET_COL}")
print(f"分组列（色谱柱）：{GROUP_COL}")

# 检查关键列是否存在
if TARGET_COL not in df.columns:
    print(f"\n❌ 错误：目标列 '{TARGET_COL}' 不存在！请核对保留因子列名")
elif GROUP_COL not in df.columns:
    print(f"\n❌ 错误：分组列 '{GROUP_COL}' 不存在！请核对色谱柱列名")
else:
    print(f"\n✅ 列名配置正确！")
    print(f"色谱柱类型分布：\n{df[GROUP_COL].value_counts()}")
    print(f"保留因子描述统计：\n{df[TARGET_COL].describe()}")

FileNotFoundError: [Errno 2] No such file or directory: '色谱柱1和色谱柱2在30度下分开做pca分析.xlsx'